<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند. 

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 
</p>
</div>

In [3]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Union
import json

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید. 
</p>
</div>

In [4]:
# load the dataset
corpus_df = pd.read_csv("AI & Bioinformatics/data.csv")[["paperId", "title", "abstract"]]
validation_data = json.load(open("AI & Bioinformatics/validation.json", 'r'))

In [5]:
corpus_df

,paperId,title,abstract
0,40ea606185b59cd07b456cb1022d64bf41f5538d,Analysis on ground surface in ultrasonic face ...,NaN
1,597c2d96c45e8ad83fc08e5d464d266b68f873ed,Measuring SARS-CoV-2 neutralizing antibody act...,The emergence of SARS-CoV-2 has created a need...
2,afbf330f0180320deff12fe42ded4f087b4a1811,A variational model for disocclusion,In this paper we study a variational approach ...
3,fa57e1cbaa46211499749cc75172e9ced26ad539,Helicoidally Arranged Polyacrylonitrile Fiber-...,"In this study, we demonstrate the use of paral..."
4,8780ce7c0d09d36d293def6be23e64b12c644169,The Interaction of SIRT4 and Calreticulin duri...,NaN
...,...,...,...
6180,dd837f59449129861086e5a96b4819a9a697c541,pH-Sensitive Biomaterials for Drug Delivery,The development of precise and personalized me...
6181,41156a34679d10200ad4bc5ba3fb391c484417ab,Glutathione-decorated fluorescent carbon quant...,NaN
6182,7cc4ecd09745b8aec00bfe07618db4d6be05cca2,High Flow Oxygen Analyzer Design on High Flow ...,High Flow Nasal Canulla (HFNC) is a technique ...
6183,188a93c03c4db17c487ccc9419864d341acf0bb9,Drug mechanism enrichment analysis improves pr...,BACKGROUND There is a pressing need for improv...


In [6]:
# drop rows which both title and abstract are nan
corpus_df = corpus_df.dropna(subset=['title', 'abstract'], how='all')
corpus_df = corpus_df.fillna("")

In [7]:
corpus_df.head()

,paperId,title,abstract
0,40ea606185b59cd07b456cb1022d64bf41f5538d,Analysis on ground surface in ultrasonic face ...,
1,597c2d96c45e8ad83fc08e5d464d266b68f873ed,Measuring SARS-CoV-2 neutralizing antibody act...,The emergence of SARS-CoV-2 has created a need...
2,afbf330f0180320deff12fe42ded4f087b4a1811,A variational model for disocclusion,In this paper we study a variational approach ...
3,fa57e1cbaa46211499749cc75172e9ced26ad539,Helicoidally Arranged Polyacrylonitrile Fiber-...,"In this study, we demonstrate the use of paral..."
4,8780ce7c0d09d36d293def6be23e64b12c644169,The Interaction of SIRT4 and Calreticulin duri...,


In [74]:
# preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize, wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import re
import string


class Preprocessor:

    def __init__(self, stopwords_path=None):
        # Create a variable of stop words.
        if stopwords_path:
            with open(stopwords_path, "r") as f:
                self.stopwords = f.read().split("\n")
        else:
            self.stopwords = list(stopwords.words("english"))

    def preprocess(self, text):
        # The main function of the class.
        text = self.normalize(text)
        text = self.remove_links(text)
        tokens = self.word_tokenize(text)
        words = self.remove_punctuations(tokens)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text="", words=[]):
        # Normalize text (lower case, stemming, lemmatization, etc.)

        if not words:
            # tokenize the text
            words = wordpunct_tokenize(text)

        # stemming
        # stemmer = SnowballStemmer("english")
        # stemmed = [stemmer.stem(word) for word in words]

        # lemmitization
        lemmatizer = WordNetLemmatizer()
        # lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]
        lemmatized = [lemmatizer.lemmatize(word) for word in words]

        normalized_text = " ".join(lemmatized)

        return normalized_text.lower()

    def remove_links(self, text):
        # Remove links
        text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
        return text

    def remove_punctuations(self, tokens):
        # Remove punctutaions
        words = [x for x in tokens if not re.fullmatch('[' + string.punctuation + ']+', x)]
        return words

    def word_tokenize(self, text):
        # Tokenize text
        return word_tokenize(text)

    def remove_stopwords(self, words):
        # Remove stopwords
        words = [word for word in words if word not in self.stopwords]
        return words


In [75]:
corpus = {}
for doc in corpus_df.iterrows():
    corpus[doc[1]["paperId"]] = {"title": doc[1]["title"], "abstract": doc[1]["abstract"]}

In [76]:
preprocessor = Preprocessor()
for docId in corpus:
    corpus[docId]["title"] = preprocessor.preprocess(corpus[docId]["title"])
    corpus[docId]["abstract"] = preprocessor.preprocess(corpus[docId]["abstract"])

<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید. 
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید. 
</p>
</div>

In [214]:
def construct_positional_indexes(corpus: dict) -> dict:
    """
    Constructs positional indexes for the processed data by inserting words into a trie and creating positional indexes and posting lists.

    Parameters
    ----------
    corpus : dict
        The processed data.

    Returns
    -------
    dict
        The constructed positional indexes.
    """

    positional_index = dict()
    for docId in corpus:
        for i, t in enumerate(corpus[docId]["title"]):
            if t not in positional_index:
                positional_index[t] = {'title': dict(), 'abstract': dict(), "tf": dict()} 
            if docId not in positional_index[t]["title"]:
                positional_index[t]["title"][docId] = []
            if docId not in positional_index[t]["tf"]:
                positional_index[t]["tf"][docId] = 0
            positional_index[t]["title"][docId].append(i)
            positional_index[t]["tf"][docId] += 1
        for i, a in enumerate(corpus[docId]["abstract"]):
            if a not in positional_index:
                positional_index[a] = {'title': dict(), 'abstract': dict(), "tf": dict()}
            if docId not in positional_index[a]["abstract"]:
                positional_index[a]["abstract"][docId] = []
            if docId not in positional_index[a]["tf"]:
                positional_index[a]["tf"][docId] = 0
            positional_index[a]["abstract"][docId].append(i)
            positional_index[a]["tf"][docId] += 1


    return positional_index


docs = construct_positional_indexes(corpus)

<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر 
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم. 

در مدل‌های احتمالاتی اساس کار محاسبه 
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش 
Okapi25
طراحی می‌کنید. 
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [177]:
def get_tf(token: str, positional_index: dict, doc_id: str) -> tuple[int, int]:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """

    title_tf = len(positional_index[token]["title"].get(doc_id, []))
    abstract_tf = len(positional_index[token]["abstract"].get(doc_id, []))

    return title_tf, abstract_tf


def get_tf_title(token: str, positional_index: dict, doc_id: str) -> tuple[int, int]:
    return len(positional_index[token]["title"].get(doc_id, []))


def get_tf_abstract(token: str, positional_index: dict, doc_id: str) -> tuple[int, int]:
    return len(positional_index[token]["abstract"].get(doc_id, []))


def get_query_tf(token: str, query: list[str]) -> int:
    tf = 0
    for term in query:
        if token == term:
            tf += 1
    return tf


get_tf('deep', docs, '40ea606185b59cd07b456cb1022d64bf41f5538d')

(0, 0)

In [188]:
def get_idf(token: str, positional_index: dict, corpus: dict) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.

    Returns
    -------
    float
        The IDF of the given token.
    """

    # get the document frequency
    title_docs = set(positional_index[token]["title"].keys())
    abstract_docs = set(positional_index[token]["abstract"].keys())

    df = len(title_docs.union(abstract_docs))
    
    
    
    return np.log(len(corpus) / df)


def get_idf_title(token: str, positional_index: dict, corpus: dict) -> float:
    idf = np.log(len(corpus) / len(positional_index[token]["title"].keys()))
    return idf

def get_idf_abstract(token: str, positional_index: dict, corpus: dict) -> float:
    idf = np.log(len(corpus) / len(positional_index[token]["abstract"].keys()))
    return idf


get_idf('deep', docs, corpus)

2.4873356041034036

<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [189]:
def logarithmic_tf(tf: int):
    return 1 if tf == 0 else 1 + np.log(tf)

In [190]:
def generate_tfidf_list(corpus: dict, positional_index: dict) -> list[tuple[str, dict[str, float]]]:
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.

    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.

    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """

    # TODO: Write the function to generate the list of dictionaries with TF-IDF scores.
    tfidf_list = []
    for docId in corpus:
        tfidf_dict = {"title": {}, "abstract": {}}
        for token in set(corpus[docId]["title"]).union(set(corpus[docId]["abstract"])):
            title_tf, abstract_tf = get_tf(token, positional_index, docId)
            title_tf, abstract_tf = logarithmic_tf(title_tf), logarithmic_tf(abstract_tf)
            idf = get_idf(token, positional_index, corpus)
            tfidf_dict["title"][token] = title_tf * idf
            tfidf_dict["abstract"][token] = abstract_tf * idf
        tfidf_list.append((docId, tfidf_dict))
    return tfidf_list


tfidf = generate_tfidf_list(corpus, docs)

In [191]:
def show_document_vector(vector: list[dict[str, float]], doc_index: int) -> None:
    """
    Prints the non-zero weights and corresponding terms for a document represented as a vector.

    Parameters
    ----------
    vector : list of dict of str:float
        The list of document vectors, where each document vector is a dictionary with term weights.
    doc_id : int
        The document index for which to display the vector.

    Returns
    -------
    None
    """
    vector_list = [(term, weight) for term, weight in vector[doc_index][1]["title"].items() if weight > 0]
    vector_list.sort(key=lambda x: x[1], reverse=True)
    length = math.sqrt(sum(weight ** 2 for _, weight in vector_list))
    normalized = {term: tfidf / length for term, tfidf in vector_list}
    for term, tfidf in vector_list:
        print(f'{term.ljust(16)}\t{tfidf}\t(normalized: {normalized[term]})')


show_document_vector(tfidf, 0)

carbide         	8.729558869558568	(normalized: 0.39992015272509734)
sic             	8.036411688998623	(normalized: 0.3681655669032228)
grinding        	7.343264508438678	(normalized: 0.3364109810813482)
ultrasonic      	6.937799400330514	(normalized: 0.3178357391496199)
vibration       	6.164609512097032	(normalized: 0.2824142221167127)
ceramic         	6.021508668456359	(normalized: 0.275858460010172)
silicon         	6.021508668456359	(normalized: 0.275858460010172)
minor           	5.594064653629419	(normalized: 0.25627631637091813)
amplitude       	5.551505039210623	(normalized: 0.25432656750587074)
ground          	4.87941126784851	(normalized: 0.22353648432927206)
face            	4.618685005385258	(normalized: 0.21159204495246078)
surface         	3.587895313055909	(normalized: 0.16436931842714497)
analysis        	1.7264934107721068	(normalized: 0.07909443292977873)


<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش 
term-at-a-time
و هم به روش 
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن 
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

In [192]:
def normalize_score(vector: dict):
    norm = np.sqrt(np.sum([v for k, v in vector.items()]))
    return {token: score / norm for token, score in vector.items()}

In [193]:
# normalize the title and abstract scores
corpus_tfidf = []
for docId, vector in tfidf:
    title_vector, abstract_vector = vector["title"], vector["abstract"]
    corpus_tfidf.append((docId, {"title": normalize_score(title_vector), "abstract": normalize_score(abstract_vector)}))

In [185]:
def logarithmic_tf_vector(tf_vector: dict):
    return {token: 1 + np.log(tf) for token, tf in tf_vector.items()}

In [256]:
def ltc_lnc_tat(query_tf: str, weight: float, corpus_tfidf: list[tuple[str, dict]]):
    normalized_query_score = normalize_score(logarithmic_tf_vector(query_tf))
    term_scores = {}
    for term, score in normalized_query_score.items():
        term_score = {}
        for docId, vector in corpus_tfidf:
            title_vector, abstract_vector = vector["title"], vector["abstract"]
            term_score[docId] = score * (weight * title_vector.get(term, 0)  + (1 - weight) * abstract_vector.get(term, 0))
        term_scores[term] = term_score
        
    doc_scores = []
    for docId, _ in corpus_tfidf:
        doc_score = 0.0
        for term, scores in term_scores.items():
            doc_score += scores[docId]
        doc_scores.append((docId, doc_score))
        
    doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)
    return doc_scores
    
    

def ltc_lnc_dat(query_tf: dict, weight: float, corpus_tfidf: list[tuple[str, dict]]):
    # normalize the scores
    normalized_query_score = normalize_score(logarithmic_tf_vector(query_tf))
    doc_scores = []
    for docId, vector in corpus_tfidf:
        title_vector, abstract_vector = vector["title"], vector["abstract"]
        title_score = sum(normalized_query_score[token] * title_vector.get(token, 0) for token in query_tf)
        abstract_score = sum(normalized_query_score[token] * abstract_vector.get(token, 0) for token in query_tf)
        doc_scores.append((docId, weight * title_score + (1 - weight) * abstract_score))
    
    doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)
    return doc_scores


def okapi25(query: list[str], weight: float, positional_index: dict, corpus: dict, k1: float = 1.2, b: float = 0.5):
    doc_scores = []
    title_avg_len = np.mean([len(doc["title"]) for docId, doc in corpus.items()])
    abstract_avg_len = np.mean([len(doc["abstract"]) for docId, doc in corpus.items()])
    for docId in corpus:
        title_rsvd = 0.0
        for term in query:
            tf = get_tf_title(term, positional_index, docId)
            idf = get_idf_title(term, positional_index, corpus)
            title_rsvd += idf * (((k1 + 1) * tf) / (k1 * ((1 - b) + b * len(corpus[docId]["title"]) / title_avg_len) + tf))
        abstract_rsvd = 0.0
        for term in query:
            tf = get_tf_abstract(term, positional_index, docId)
            idf = get_idf_abstract(term, positional_index, corpus)
            abstract_rsvd += idf * (((k1 + 1) * tf) / (k1 * ((1 - b) + b * len(corpus[docId]["abstract"]) / abstract_avg_len) + tf))

        doc_scores.append((docId, weight * title_rsvd + (1 - weight) * abstract_rsvd))
        
    doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)
    return doc_scores


def search(query: str, max_result_count: int, method: str = 'ltc-lnc-tat', weight: float = 0.5,
           positional_index: dict = docs, corpus: dict = corpus, corpus_tfidf: list[dict] = tfidf):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    method: 'ltc-lnc-tat' or 'ltc-lnc-dat' or 'okapi25'

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """

    query = Preprocessor().preprocess(query)
    query_tf = {token: get_query_tf(token, query) for token in query}
    if method == "ltc-lnc-tat":
        result = ltc_lnc_tat(query_tf, weight, corpus_tfidf)
    elif method == "ltc-lnc-dat":
        result = ltc_lnc_dat(query_tf, weight, corpus_tfidf)
    elif method == "okapi25":
        result = okapi25(query, weight, positional_index, corpus)
    else:
        raise Exception(f"The method {method} is not implemented!")
    
    if max_result_count == -1:
        return [x[0] for x in result]
    return [x[0] for x in result[:max_result_count]]

In [257]:
print(search("face analysis with minor changes", 5, "ltc-lnc-tat"))
print(search("face analysis with minor changes", 5, "ltc-lnc-dat"))
print(search("face analysis with minor changes", 10, "okapi25"))

['40ea606185b59cd07b456cb1022d64bf41f5538d', 'e42d84a6e635cc8265820cc0ac7728c1171736df', '19697a2ef70f53bf235712b5e837993e623d5341', '342d2704b3b7cc5867864677be814d270e57dd0f', 'b72be0db4e827d3e0e4b52b9085d9d608ecec60c']
['40ea606185b59cd07b456cb1022d64bf41f5538d', 'e42d84a6e635cc8265820cc0ac7728c1171736df', '19697a2ef70f53bf235712b5e837993e623d5341', '342d2704b3b7cc5867864677be814d270e57dd0f', 'b72be0db4e827d3e0e4b52b9085d9d608ecec60c']
['721d0c5a193fb5b5c357ca56e20d28e1ef4c9ca7', '5fa712fb6809e876ad5a967c7f06c7e4e521c5b7', '67c4528de75ae743d23a4eea5c03316a7a8cf9d1', 'd35f269c9bb0ccde4b7f71b0d3224cf1e9e55fb5', '40ea606185b59cd07b456cb1022d64bf41f5538d', '4668306bc74576645c9cbe39a08f5f51a9bce4f4', '5ea34ef45e1a5ad2720d2874e5c85c26e8930176', 'd5f1886482893adf2f61e3b344d5483c86792ab5', '58f842afbc5944371fddc412ee8cd67451199abb', '55278198d72e6efcdeab9d10a2912fd58a9d48e7']


<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل 
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک 
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه 
smoothing 
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد. 
</p>
</div>

In [258]:
doc_len = {docId: len(doc["title"]) + len(doc["abstract"]) for docId, doc in corpus.items()}
total_tf = sum(term_total_tf.values())

In [259]:
def lm_search(query: str, max_result_count: int = -1, positional_index: dict = docs, corpus: dict = corpus, doc_len: dict = doc_len, _lambda: float= 0.9):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    query = Preprocessor().preprocess(query)
    doc_scores = []
    for docId, doc in corpus.items():
        score = 1
        for term in query:
            score *= (_lambda * positional_index[term]["tf"].get(docId, 0) / doc_len[docId]  + (1 - _lambda) *
                      sum([tf for docId, tf in positional_index[term]["tf"].items()]) / total_tf
                      )
            
        
        doc_scores.append((docId, score))
    
    doc_scores = sorted(doc_scores, key=lambda x: x[1], reverse=True)
    
    if max_result_count == -1:
        return [x[0] for x in doc_scores]
    
    return [x[0] for x in doc_scores[:max_result_count]]

In [260]:
lm_search("face analysis with minor changes", 5)

['40ea606185b59cd07b456cb1022d64bf41f5538d',
 'd75de30c90ae8cd3326175f82d77add28ccc53fe',
 'e42d84a6e635cc8265820cc0ac7728c1171736df',
 '19697a2ef70f53bf235712b5e837993e623d5341',
 '721d0c5a193fb5b5c357ca56e20d28e1ef4c9ca7']

<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی 
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید. 
</p>
</div>

In [304]:
class Evaluation:
    def __init__(self, actual, predicts):
        self.actual = actual
        self.queries = predicts

    def evaluate(self, require_scores):
        """
        Prints require scores for actual and predicts array

        Parameters
        ----------
        require_scores : List[str]
            Scores required to calculated

        Returns
        -------
        """
        for rs in require_scores:
            if rs == 'precision':
                    precision = self.precision()
                    print(f'Precision: {precision}')
            elif rs =='recall':
                    recall = self.recall()
                    print(f'Recall: {recall}')
            elif rs == 'f1':
                f1 = self.f1()
                print(f'F1: {f1}')
            elif rs == 'map':
                _map = self.map()
                print(f'MAP: {_map}')
            elif rs == 'ndcg':
                ndcg = self.ndcg()
                print(f'NDCG: {ndcg}')
            elif rs == 'mrr':
                mrr = self.mrr()
                print(f'MRR: {mrr}')
            else:
                raise ValueError(f'{rs} is not a valid score')
        
            

    def precision(self):
        """
        Calculates the precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The precision of the predicted results
        """
        
        precision = 0
        for q in self.queries:
            tp = len(set(self.actual[q]).intersection(set(self.queries[q])))
            # print(self.queries[i], self.actual[i])
            fp = len(set(self.queries[q]).difference(set(self.actual[q])))
            precision += tp / (tp + fp)

        return precision / len(self.queries)

    def recall(self):
        """
        Calculates the recall of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The recall of the predicted results
        """
        
        recall = 0
        for q in self.queries:
            tp = len(set(self.actual[q]).intersection(set(self.queries[q])))
            fn = len(set(self.actual[q]).difference(set(self.queries[q])))
            recall += tp / (tp + fn)
            
        return recall / len(self.queries)
        

    def f1(self):
        """
        Calculates the F1 score of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The F1 score of the predicted results
        """
        f1_score = (2 * self.precision() * self.recall()) / (self.precision() + self.recall())

        return f1_score
    
    def P_at_K(self, k, query):
        """
        Computes precision at k (P@k)

        Parameters
        ----------
        k : int
            The k value to compute P@k for
        query: list
            The query results

        Returns
        -------
        float
            The P@k value
        """
        
        relevant = set(self.actual[query][:k]).intersection(set(self.queries[query][:k]))
        return len(relevant) / k
        

    def map(self):
        """
        Calculates the Mean Average Precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The Mean Average Precision of the predicted results
        """
        map_score = 0.0
        for query in self.queries:
            num_relevant = len(set(self.actual[query]).intersection(set(self.queries[query])))
            if num_relevant == 0:
                continue
            sum_precision = 0.0
            for k in range(1, len(self.queries[query]) + 1):
                sum_precision += self.P_at_K(k, query)
            map_score += sum_precision / num_relevant
            
        return map_score / len(self.queries)

    def ndcg(self):
        """
        Calculates the Normalized Discounted Cumulative Gain (NDCG) of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The NDCG of the predicted results
        """
        nDCG = 0.0
        for query in self.queries:
            predicted, actual = self.queries[query], self.actual[query]
            relevance = {doc: i + 1 for i, doc in enumerate(actual[::-1])}
            rel = [relevance.get(doc, 0) for doc in predicted]
            DCG = sum(rel[k] / math.log2(k + 2) for k in range(len(predicted)))
            sorted_rel = sorted(rel, reverse=True)
            IDCG = sum(sorted_rel[k] / math.log2(k + 2) for k in range(len(predicted)))
            nDCG += 0 if IDCG == 0 else DCG / IDCG
        nDCG /= len(self.actual)
        return nDCG
    
    def mrr(self):
        """
        Calculates the Mean Reciprocal Rank of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The MRR of the predicted results
        """
        MRR = 0.0
        for query in self.queries:
            predicted, actual = self.queries[query], self.actual[query]
            actual_set = set(actual)
            MRR += next((1.0 / (i + 1) for i, doc in enumerate(predicted) if doc in actual_set), 0)
        MRR /= len(self.actual)
        return MRR

In [286]:
require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

# Read actuals and queries from validation.json file

validation_data = json.load(open("AI & Bioinformatics/validation.json"))

actuals = validation_data
queries = validation_data.keys()

In [305]:
predicts = {query: search(query, 10, method='ltc-lnc-dat') for query in queries}

eval_vsm = Evaluation(actuals, predicts)
eval_vsm.evaluate(require_scores)

Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1: 0.6666666666666666
MAP: 0.52317791005291
NDCG: 0.6870322770890116
MRR: 0.7222222222222222


In [306]:
predicts = {query: search(query, 10, method='ltc-lnc-tat') for query in queries}

eval_vsm = Evaluation(actuals, predicts)
eval_vsm.evaluate(require_scores)

Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1: 0.6666666666666666
MAP: 0.52317791005291
NDCG: 0.6870322770890116
MRR: 0.7222222222222222


In [307]:
predicts = {query: search(query, 10, method='okapi25') for query in queries}

eval_bm = Evaluation(actuals, predicts)
eval_bm.evaluate(require_scores)

Precision: 0.6833333333333332
Recall: 0.6833333333333332
F1: 0.6833333333333332
MAP: 0.6758003432854623
NDCG: 0.8869536272761072
MRR: 1.0


In [308]:
predicts = {query: lm_search(query, 10) for query in queries}

eval_lm = Evaluation(actuals, predicts)
eval_lm.evaluate(require_scores)

Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1: 0.6666666666666666
MAP: 0.8086055996472662
NDCG: 0.7812206250311635
MRR: 0.7833333333333333
